# Project Group - 15

Members: 
Sören Burghardt, 
Sarah Blanc, 
Allan Guzmann, 
Lars van den Berg, 
Simon Schreders

Student numbers: 
5861012, 
5854830, 
5718619,
5626668,
5878845

# Research Objective

What are the effects on aviation that COVID-19  caused at the Amsterdam Schiphol Airport and its most frequented destinations for freight and passenger transport?   

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

COVID 19 Data from the Netherlands and most frequented destinations (RIVM, our world open data) 

Flight data passengers (Eu Stat)

Flight data freight (EU Stat)

OD Data for Airport Amsterdam (EU Stat)


# Data Pipeline